## Codon Table Class

This class instantiates codon tables with arbitrary, user defined decoding strategies.

In [2]:
# import necessary modules
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import Bio.Data.CodonTable
import networkx as nx
from codonUtils import utils
from codonoptimizer import MonteCarlo
from codonTable import codonTable
import random
import pickle

## Standard Code and Biological Dictionaries

The exact ordering of amino acids and stop codons in order to represent the Standard Code in a pythonic way (e.g. defined by iterating over zipped elements) pulled from NCBI (https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi#SG1).

## Codon Utilities Module

This module was written to wrap a set of generally useful definitions and methods. These include 
- the Standard Codon table
- the set of NTPs and amino acids
- some physicochemical scales (Polar Requirement Scale and Kyte Doolittle Hydropathy)
- codon table block structures
- methods to count amino acid degeneracy
- methods to find adjacency of both codons and residues in a network
- methods to interconvert Codon Table structure to/from representations in block structure form
- method to check the validity of a codon table against a specified block structure

The module is fully documented, as in calling help(codonUtils.utils) will return full documentation of associated attributes and methods.

In [ ]:
help(utils)

## Visualization Object for Codon Tables

This class is designed to take dictionaries representing triplet codon tables and visualizing them in 3D space. For simplicity, the default visualization metric used is the 1982 hydrophobicity scale developed by Kyte and Doolittle (1).

In [ ]:
help(codonTable)

In [ ]:
# Demonstrate CodonTable visualization
%matplotlib inline
test = codonTable()
fig = test.plot3d('Standard Codon Table: Node Color=Hydropathy', )
fig2 = test.plotGraph('Standard Codon Table: Node Color=Residue Degeneracy', node_val='count')

## Monte Carlo Simulation Class
This is the class for handling walking through Codon Table space to optimize for a user defined cost function. Defaults to self.cost() if not given. Any uses defined cost function should accept a Python dict representing the codon table (C --> AA) as an input and output a float to be maximized over (must set self.run() to minimization otherwise). Optionally should accept codonTable class as input. Uses the Polar Requirement Scale (3) as a measure of codon similarity.

In [37]:
help(MonteCarlo)

Help on class MonteCarlo in module codonoptimizer:

class MonteCarlo(builtins.object)
 |  MonteCarlo: A class designed to optimize a codon table given an
 |  arbitrary objective function to minimize/maximize
 |  
 |  Methods defined here:
 |  
 |  GDA(self, dW=None, W=0, W_stop=inf, maxIter=1000, preserveBlock=False, preserveStop=False)
 |      The Great Deluge Algorithm for optimizing an objective function over
 |      codon table space
 |      
 |      Parameters
 |      ----------
 |      - float dW=None: represents the rate of change (rain flux) of min
 |          allowable energy (water level); if None, initialized to 1% of
 |          cost(self.table)
 |      - float W=0: represents the initial minimal acceptable energy level
 |          (water level); defaults to 0
 |      - float W_stop=inf: represents the maximum allowable water level; stops
 |          algorithm when W > W_stop; defaults to an infinte value
 |      - int maxIter=1000: represents the maximum number of iteratio

In [7]:
sim = MonteCarlo()
%timeit sim.tableShuffle(sim.table)

44.5 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [5]:
sim = MonteCarlo()
count = 0
for i in range(1000):
    table = sim.tableShuffle(sim.table)
    if not utils.checkBlock(table, sim.blockStruct):
        count+=1
count

0

In [18]:
x = np.zeros(100)
x[0:5] = np.arange(0,5)
x[5:] = x[4]
x

array([ 0.,  1.,  2.,  3.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.])

In [ ]:
# Debugging
sim = MonteCarlo()
table = sim.table
counts = sim.resiCounts
connect = sim.connectivity
blockStruct = utils.naturalBlock
blocks = utils.tableToBlocks(table, blockStruct)
okay = set()
for AA, count in counts.items():
    if count > 1:
        okay.add(AA)

blocks

## References
1. Kyte, J. & Doolittle, R. F. A simple method for displaying the hydropathic character of a protein. J. Mol. Biol. 157, 105–132 (1982).
1. Novozhilov, A. S., Wolf, Y. I. & Koonin, E. V. Evolution of the genetic code: partial optimization of a random code for robustness to translation error in a rugged fitness landscape. Biol. Direct 2, 24 (2007).
1. Woese, C. R., Dugre, D. H., Saxinger, W. C. & Dugre, S. A. The Molecular Basis for the Genetic Code. Proc. Natl. Acad. Sci. 55, 966–974 (1966).